In [ ]:
import geopy
from geopy.geocoders import Nominatim
import requests
import folium
import pandas as pd
from time import sleep
from copy import deepcopy
from itertools import combinations

pd.options.mode.chained_assignment = None

## LOAD DATA

In [ ]:
with open('cast_types.txt', 'r') as file:
    cast_types = [f.strip('\n') for f in file.readlines()]

In [ ]:
data = {}
for cast_ in cast_types:
    data[cast_] = pd.read_csv(f'data/{cast_}.csv').set_index('name')

## DIRECTIONS API

In [ ]:
with open('gitignore/api_key.txt') as fh:
    api_key = fh.read()

In [ ]:
geolocator = Nominatim(user_agent="Transport_planner")

## CREATE OBJECT

In [ ]:
class TransportPlanner(object):
    
    def __init__(self, supervisors, drivers, cast_members, destinations, geolocator=geolocator, api_key=api_key):
        self.supervisors = supervisors
        self.drivers = drivers
        self.cast_members = cast_members
        self.destinations = destinations
        self.all_locs = [supervisors, drivers, cast_members, destinations]
        self.geolocator = geolocator
        self.api_key = api_key
        self.capacities = drivers['capacity']
        self.dest_counts = cast_members['destination'].value_counts()
        with open('cast_types.txt', 'r') as file:
            self.cast_types = [f.strip('\n') for f in file.readlines()] 
        
    def __str__(self):
        return('This is a transport planner object.')
    
    
    def get_locations(self):
        self.locations = {}
        for df in self.all_locs:
            for ind, row in df.iterrows():
                try:
                    self.locations[ind] = self.geolocator.geocode(row['address'])
                except:
                    continue
    
    def get_coordinates(self):
        self.coords = pd.DataFrame({k: (v.longitude, v.latitude) for k, v in self.locations.items()}).T
    
    def get_directions_locations(self, loc_1, loc_2):
        # get direction as a geojson
        url = f"https://api.openrouteservice.org/v2/directions/driving-car?api_key={self.api_key}&start={str(loc_1.longitude)},{str(loc_1.latitude)}&end={str(loc_2.longitude)},{str(loc_2.latitude)}"
        r = requests.get(url)
        return r.json()
    
    def get_directions_coords(self, coords_1, coords_2):
        # get direction as a geojson
        url = f"https://api.openrouteservice.org/v2/directions/driving-car?api_key={self.api_key}&start={str(coords_1[0])},{str(coords_1[1])}&end={str(coords_2[0])},{str(coords_2[1])}"
        r = requests.get(url)
        return r.json()
    
    def get_duration(self, loc_1, loc_2, loc=True):
        # return time of route plan in seconds
        if loc:
            geojson = self.get_directions_locations(loc_1, loc_2)
        else:
            geojson = self.get_directions_coords(loc_1, loc_2)
        return geojson, geojson['features'][0]['properties']['segments'][0]['duration']
    
    def get_distance_matrix(self):
        dist_table = {}
        geojsons = {}
        counter = 0
        for key, loc in self.locations.items():
            dist_table[key] = {}
            geojsons[key] = {}
            for keyy, locc in self.locations.items():
                
                # for rate limit
                counter += 1
                if counter > 39:
                    sleep(60)
                    counter = 0
                else:
                    pass
                
                # get durations
                if key == keyy:
                    dur = 0
                    geojson = {'features': None}
                else:
                    geojson, dur = self.get_duration(loc, locc, True)
                geojsons[key][keyy] = geojson
                dist_table[key][keyy] = dur
                
        self.dist_table = dist_table
        self.geojsons = geojsons
        return None
    
    def get_next_cast_member(self, start, relevant_cast_names):
        relevant_dists = {k: v for k, v in self.dist_table[start].items() if k in relevant_cast_names}
        return min(relevant_dists, key=relevant_dists.get)
    
    def get_optimal_route(self, dest, capacity, cast_done):
        driving_order = [dest]
        cast_solved = []
        start = dest
        relevant_cast_names = self.cast_members[tp.cast_members['destination'] == dest].index
        relevant_cast_names = [f for f in relevant_cast_names if not f in cast_done]
        while len(driving_order) < capacity + 1:
            if len(relevant_cast_names) == 0:
                return driving_order
            next_cast = self.get_next_cast_member(start, relevant_cast_names)
            driving_order.append(next_cast)
            relevant_cast_names = [f for f in relevant_cast_names if not f in driving_order]
            start = next_cast
        return driving_order
    
    def allocate_driver(self, driving_order, cap, drivers_done):
        relevant_drivers = [f for f in self.capacities.index if self.capacities[f] == cap]
        relevant_drivers = [f for f in relevant_drivers if not f in drivers_done]
        driver = self.get_next_cast_member(driving_order[-1], relevant_drivers)
        ret = driving_order + [driver]
        return ret
    
    def optimize_destinations(self):
        dest_cast_counts = self.cast_members['destination'].value_counts()
        dest_order = dest_cast_counts.sort_values().index
        drivers = {}
        cast_done = []
        drivers_done = []
        for dest in dest_order:
            if dest_cast_counts[dest] in self.capacities.tolist():
                driving_order = self.get_optimal_route(dest, dest_cast_counts[dest], cast_done)
                final_driving_order = self.allocate_driver(driving_order, dest_cast_counts[dest], drivers_done)
                cast_done += driving_order[1:]
                drivers_done += [final_driving_order[-1]]
                drivers[final_driving_order[-1]] = final_driving_order[::-1]
            elif dest_cast_counts[dest] < tp.capacities.min():
                driving_order = self.get_optimal_route(dest, tp.capacities.min(), cast_done)
                final_driving_order = self.allocate_driver(driving_order, tp.capacities.min(), drivers_done)
                cast_done += driving_order[1:]
                drivers_done += [final_driving_order[-1]]
                drivers[final_driving_order[-1]] = final_driving_order[::-1]
            else:
                STOP = False
                driver_capacities = self.drivers.drop(drivers_done)['capacity'].tolist()
                sorted_driver_capacities = sorted(driver_capacities)[::-1]
                optimal_capacities = None
                for i in range(len(driver_capacities)):
                    for comb in combinations(driver_capacities, i):
                        if sum(comb) == dest_cast_counts[dest]:
                            STOP = True
                            optimal_capacities = comb
                            break
                    if STOP:
                        break
                if not optimal_capacities:
                    for i in range(2, len(driver_capacities)):
                        for comb in combinations(driver_capacities, i):
                            if sum(comb) > dest_cast_counts[dest]:
                                STOP = True
                                optimal_capacities = comb
                                break
                for elem in comb:
                    driving_order = self.get_optimal_route(dest, elem, cast_done)
                    final_driving_order = self.allocate_driver(driving_order, elem, drivers_done)
                    cast_done += driving_order[1:]
                    drivers_done += [final_driving_order[-1]]
                    drivers[final_driving_order[-1]] = final_driving_order[::-1] 

        assert set(cast_done) == set(self.cast_members.index), 'Not all cast members have a ride!!!'
        self.driver_routes = drivers

In [ ]:
tp = TransportPlanner(*[data[f] for f in cast_types])

In [ ]:
tp.get_locations()

In [ ]:
tp.get_coordinates()

In [ ]:
tp.get_distance_matrix()

In [ ]:
tp.optimize_destinations()

In [ ]:
## map creation method - maps will be attached and can be opened in browser!!!
tp.itineraries = {}
for driver in tp.driver_routes.keys():
    tp.itineraries[driver] = folium.Map(location=(pd.np.median(tp.coords[1]), pd.np.median(tp.coords[0])), zoom_start=12)
    fg = folium.FeatureGroup(name='Stops')
    for i, stop in enumerate(tp.driver_routes[driver]):
        if not i == len(tp.driver_routes[driver])-1:
            folium.GeoJson(tp.geojsons[tp.driver_routes[driver][i]][tp.driver_routes[driver][i+1]]).add_to(tp.itineraries[driver])
            if i == 0:
                fg.add_child(folium.Marker(location = [tp.coords.loc[stop, 1], tp.coords.loc[stop, 0]],
                                           popup='START', 
                                           icon=folium.Icon(color='lightblue')))
            else:
                fg.add_child(folium.Marker(location = [tp.coords.loc[stop, 1], tp.coords.loc[stop, 0]],
                                           popup=f"STOP {str(i)}",
                                           icon=folium.Icon(color='blue')))
        else:
            fg.add_child(folium.Marker(location = [tp.coords.loc[stop, 1], tp.coords.loc[stop, 0]],
                                           popup="FINAL DESTINATION",
                                           icon=folium.Icon(color='darkblue')))
    tp.itineraries[driver].add_child(fg)
        
        

In [ ]:
help(folium.Icon)

In [ ]:
tp.driver_routes

In [ ]:
tp.itineraries['Feri']

In [ ]:
m = folium.Map(location = (pd.np.median(tp.coords[1]), pd.np.median(tp.coords[0])), zoom_start=12)

In [ ]:
folium.GeoJson(tp.geojsons['Dani']['Boci'], name='1').add_to(m)
folium.GeoJson(tp.geojsons['Boci']['Coci'], name='2').add_to(m)
folium.GeoJson(tp.geojsons['Coci']['Zoli'], name='3').add_to(m)
folium.GeoJson(tp.geojsons['Zoli']['Dressing'], name='4').add_to(m)

In [ ]:
m

In [ ]:
tp.geojsons['Dani']['Boci']

In [ ]:
dest = tp.destinations.index[0]

In [ ]:
def optimize_destination(dest):
    cast_done = []
    drivers = {}

def allocate_driver(driving_order, cap, drivers_done):
    relevant_drivers = [f for f in tp.capacities.index if tp.capacities[f] == cap]
    relevant_drivers = [f for f in relevant_drivers if not f in drivers_done]
    driver = get_next_cast_member(driving_order[-1], relevant_drivers)
    ret = driving_order + [driver]
    return ret

def get_optimal_route(dest, capacity, cast_done):
    driving_order = [dest]
    cast_solved = []
    start = dest
    relevant_cast_names = tp.cast_members[tp.cast_members['destination'] == dest].index
    relevant_cast_names = [f for f in relevant_cast_names if not f in cast_done]
    while len(driving_order) < capacity + 1:
        if len(relevant_cast_names) == 0:
            return driving_order
        next_cast = get_next_cast_member(start, relevant_cast_names)
        driving_order.append(next_cast)
        relevant_cast_names = [f for f in relevant_cast_names if not f in driving_order]
        start = next_cast
    return driving_order
    
def get_next_cast_member(start, relevant_cast_names):
    relevant_dists = {k: v for k, v in tp.dist_table[start].items() if k in relevant_cast_names}
    return min(relevant_dists, key=relevant_dists.get)

In [ ]:
dest_cast_counts = tp.cast_members['destination'].value_counts()
dest_order = dest_cast_counts.sort_values().index

In [ ]:
dest = dest_order[0]

In [ ]:
def optimize_destination():
    dest_cast_counts = tp.cast_members['destination'].value_counts()
    dest_order = dest_cast_counts.sort_values().index
    drivers = {}
    cast_done = []
    drivers_done = []
    for dest in dest_order:
        if dest_cast_counts[dest] in tp.capacities.tolist():
            driving_order = get_optimal_route(dest, dest_cast_counts[dest], cast_done)
            final_driving_order = allocate_driver(driving_order, dest_cast_counts[dest], drivers_done)
            cast_done += driving_order[1:]
            drivers_done += [final_driving_order[-1]]
            drivers[final_driving_order[-1]] = final_driving_order[::-1]
        elif dest_cast_counts[dest] < tp.capacities.min():
            driving_order = get_optimal_route(dest, tp.capacities.min(), cast_done)
            final_driving_order = allocate_driver(driving_order, tp.capacities.min(), drivers_done)
            cast_done += driving_order[1:]
            drivers_done += [final_driving_order[-1]]
            drivers[final_driving_order[-1]] = final_driving_order[::-1]
        else:
            STOP = False
            driver_capacities = tp.drivers.drop(drivers_done)['capacity'].tolist()
            sorted_driver_capacities = sorted(driver_capacities)[::-1]
            optimal_capacities = None
            for i in range(len(driver_capacities)):
                for comb in combinations(driver_capacities, i):
                    if sum(comb) == dest_cast_counts[dest]:
                        STOP = True
                        optimal_capacities = comb
                        break
                if STOP:
                    break
            if not optimal_capacities:
                for i in range(2, len(driver_capacities)):
                    for comb in combinations(driver_capacities, i):
                        if sum(comb) > dest_cast_counts[dest]:
                            STOP = True
                            optimal_capacities = comb
                            break
            for elem in comb:
                driving_order = get_optimal_route(dest, elem, cast_done)
                final_driving_order = allocate_driver(driving_order, elem, drivers_done)
                cast_done += driving_order[1:]
                drivers_done += [final_driving_order[-1]]
                drivers[final_driving_order[-1]] = final_driving_order[::-1] 

    assert set(cast_done) == set(tp.cast_members.index), 'Not all cast members have a ride!!!'


In [ ]:
drivers

In [ ]:
tp.cast_members

In [ ]:
driving_order

In [ ]:
drivers_done

In [ ]:
drivers

In [ ]:
tp.cast_members

In [ ]:
drivers

In [ ]:
sorted(tp.drivers.drop(drivers_done)['capacity'].tolist())[::-1]

In [ ]:
drivers

In [ ]:
tp.drivers.drop(drivers_done)

In [ ]:
for comb in combinations(tp.drivers.drop(drivers_done)['capacity'].tolist(), 2):
    print(comb[0] + comb[1])

In [ ]:
final_driving_order

In [ ]:
cast_done

In [ ]:
drivers_done

In [ ]:
print(tp.destinations.loc[final_driving_order[0], 'address'])
for i in driving_order[1:]:
    print(tp.cast_members.loc[i, 'address'])
print(tp.drivers.loc[final_driving_order[-1], 'address'])

In [ ]:
relevant_drivers = [f for f in tp.capacities.index if tp.capacities[f] == len(driving_order)-1]

In [ ]:
relevant_drivers

In [ ]:
tp.cast_members.loc[ut]